<a href="https://colab.research.google.com/github/Tonb73/2025NIRS_PRAVO/blob/Develope/NIRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy3

In [ ]:
import csv
import pandas as pd
import numpy as np
import pymorphy3
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Константы


In [ ]:
max_words = 10000
random_state = 57

Загрузка входных данных


In [ ]:
data = pd.read_csv("Learn.csv",encoding = 'Windows-1251',sep = ';')
data['Desc'] = data['Desc'].fillna('')

In [ ]:
data[0:3]

,Number,Date,UID,Desc,Group,Cat
0,1060141,12.03.2021 11:33,7b5dc3bb-1dcb-4954-9f3c-c5dc9c7d45d2,Не горит лампа в коридоре,Управление домом,"Разбитые стекла, сорванные створки оконных пер..."
1,1112641,2021-05-27 14:16:23.000,8361aa95-b693-444e-a212-8e1da60e8fde,кривой дорожный знак,Дороги,Общие вопросы
2,1112656,31.05.2021 12:07,7ac9fbb2-f72f-471d-9a10-ff191b9be927,Провал колодца на пешеходной дороге,Канализация,Общие вопросы


In [ ]:
def preprocess(text, stop_words, punctuation_marks, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...', '«', '»', ';', '–', '--']
stop_words = stopwords.words("russian")
morph = pymorphy3.MorphAnalyzer()

In [ ]:
data['Preprocessed_Desc'] = data.apply(lambda row: preprocess(row['Desc'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
data[0:5]

,Number,Date,UID,Desc,Group,Cat,Preprocessed_Desc
0,1060141,12.03.2021 11:33,7b5dc3bb-1dcb-4954-9f3c-c5dc9c7d45d2,Не горит лампа в коридоре,Управление домом,"Разбитые стекла, сорванные створки оконных пер...","[гореть, лампа, коридор]"
1,1112641,2021-05-27 14:16:23.000,8361aa95-b693-444e-a212-8e1da60e8fde,кривой дорожный знак,Дороги,Общие вопросы,"[кривая, дорожный, знак]"
2,1112656,31.05.2021 12:07,7ac9fbb2-f72f-471d-9a10-ff191b9be927,Провал колодца на пешеходной дороге,Канализация,Общие вопросы,"[провал, колодец, пешеходный, дорога]"
3,1112657,31.05.2021 12:12,3777d635-b4da-4897-9b89-2ba1aa44d891,Низкие температурные параметры ГВС,ГВС,Общие вопросы,"[низкий, температурный, параметр, гвс]"
4,1112668,01.06.2021 11:17,badc0911-6201-4a4e-963e-a63a6a3d408c,запах канализации,Канализация,Общие вопросы,"[запах, канализация]"


In [ ]:
words = Counter()

In [ ]:
for txt in data['Preprocessed_Desc']:
    words.update(txt)

In [ ]:
# Словарь, отображающий слова в коды
word_to_index = dict()
# Словарь, отображающий коды в слова
index_to_word = dict()

In [ ]:
for i, word in enumerate(words.most_common(max_words - 2)):
    word_to_index[word[0]] = i + 2
    index_to_word[i + 2] = word[0]

In [ ]:
def text_to_sequence(txt, word_to_index):
    seq = []
    for word in txt:
        index = word_to_index.get(word, 1) # 1 означает неизвестное слово
        # Неизвестные слова не добавляем в выходную последовательность
        if index != 1:
            seq.append(index)
    return seq

In [ ]:
data['Sequences'] = data.apply(lambda row: text_to_sequence(row['Preprocessed_Desc'], word_to_index), axis=1)

Данные для обучения


In [ ]:
train, test = train_test_split(data, test_size=0.2)

In [ ]:
x_train_seq = train['Sequences']
z_train = train['Cat']

Данные для тестирования


In [ ]:
x_test_seq = test['Sequences']
z_test = test['Cat']


Создаем мешок слов

In [ ]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for index in sequence:
            results[i, index] += 1.
    return results

In [ ]:
x_train = vectorize_sequences(x_train_seq, max_words)

In [ ]:
x_test = vectorize_sequences(x_test_seq, max_words)

Создаем модель ML

In [ ]:
dt = DecisionTreeClassifier(class_weight='balanced')

In [ ]:
dt1 = DecisionTreeClassifier(class_weight='balanced')

Обучаем модель

In [ ]:
dt1.fit(x_train, z_train)

DecisionTreeClassifier(class_weight='balanced')

In [ ]:
outData = pd.read_csv("C.csv", encoding = 'Windows-1251',sep = ';')

In [ ]:
outData['Preprocessed_desc'] = outData.apply(lambda row: preprocess(row['Desc'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
outData['sequences'] = outData.apply(lambda row: text_to_sequence(row['Preprocessed_desc'], word_to_index), axis=1)
desc = outData['sequences']
x_desc = vectorize_sequences(desc, max_words)

In [ ]:
outData[0:3]

,Number,Date,UID,Desc,Group,Cat,Preprocessed_desc,sequences
0,1346291,01.01.2023 2:10,e87b2cfa-6159-4aa2-b789-3974a01e0b0a,Застряли в лифте,Лифты,Человек застрял в лифте,"[застрять, лифт]","[94, 37]"
1,1346298,01.01.2023 9:09,d6257b7b-d68c-449f-aef3-6c4af1a2d78f,Течь батареи в спальной,Управление домом,Сантехника,"[течь, батарея, спальная]","[2, 15, 261]"
2,1346301,01.01.2023 10:37,927aeffe-0842-4894-8ada-0a91ccdf1692,Течь сверху по стояку,Управление домом,Сантехника,"[течь, сверху, стояк]","[2, 42, 11]"


In [ ]:
predictionDt = dt.predict(x_desc)


In [ ]:
predictionDt1 = dt1.predict(x_desc)

In [ ]:
outDataDt = outData

In [ ]:
outDataDt['Cat'] = predictionDt1

In [ ]:
outDataDt[0:10]

,Number,Date,UID,Desc,Group,Cat,Preprocessed_desc,sequences
0,1346291,01.01.2023 2:10,e87b2cfa-6159-4aa2-b789-3974a01e0b0a,Застряли в лифте,Лифты,Человек застрял в лифте,"[застрять, лифт]","[94, 37]"
1,1346298,01.01.2023 9:09,d6257b7b-d68c-449f-aef3-6c4af1a2d78f,Течь батареи в спальной,Управление домом,Сантехника,"[течь, батарея, спальная]","[2, 15, 261]"
2,1346301,01.01.2023 10:37,927aeffe-0842-4894-8ada-0a91ccdf1692,Течь сверху по стояку,Управление домом,Сантехника,"[течь, сверху, стояк]","[2, 42, 11]"
3,1346303,01.01.2023 10:56,9884e4a6-2ce4-485b-913d-8e75c7779302,Течь КНС в 3 подвале,Управление домом,Канализация,"[течь, кнс, 3, подвал]","[2, 10, 40, 12]"
4,1346306,01.01.2023 11:44,4593c91a-8b75-46e4-83e5-557d730abea0,Течь кровли (забита ливневка),Управление домом,Ливнёвки,"[течь, кровля, забитый, ливневка]","[2, 25, 214, 232]"
5,1346319,01.01.2023 13:06,9884e4a6-2ce4-485b-913d-8e75c7779302,Засор в подвале 3 подъезда,Управление домом,Канализация,"[засор, подвал, 3, подъезд]","[17, 12, 40, 5]"
6,1346322,01.01.2023 13:34,a4b0fada-c9b5-4822-bc5e-5eb22a471293,Течь из колодца у 6 подъезда,Управление домом,Сантехника,"[течь, колодец, 6, подъезд]","[2, 135, 97, 5]"
7,1346323,01.01.2023 13:39,a4b0fada-c9b5-4822-bc5e-5eb22a471293,Течь отсечного вентиля ХВС,Управление домом,Сантехника,"[течь, отсечный, вентиль, хвс]","[2, 89, 61, 7]"
8,1346324,2023-01-01 13:50:46.028,1ab8b15e-b18c-4ce0-80dc-da40a029289e,Течь кровли в коридоре квартиры ручьем.,Управление домом,Кровля (Протечка),"[течь, кровля, коридор, квартира, ручьём]","[2, 25, 86, 6, 1809]"
9,1346325,01.01.2023 14:00,c7b6f761-3317-4bb4-91a3-3c71dcb1fccf,Течь ХВС сверху по стене и с потолка туалета с...,Управление домом,Сантехника,"[течь, хвс, сверху, стена, потолок, туалет, 4,...","[2, 7, 42, 59, 72, 50, 56, 23, 40, 23, 60]"


In [ ]:
columns_to_include = ["Desc","Cat"]

In [ ]:
filtered_dfDt = outDataDt[columns_to_include]
filtered_dfDt.to_csv('NewDt.csv', index=False, sep=';', encoding='Windows-1251')